In [1]:
# Import packages to use
import twint
import numpy as np
import pickle
import pandas as pd
import time
import math
from collections import Counter
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline

# Patches asyncio to allow the running of multiple event loops in Jupyter Notebooks.
# Fixes: "RuntimeError: This event loop is already running"
import nest_asyncio

nest_asyncio.apply()
print(pd.__version__)

1.1.3


In [2]:
# Finds the distance between two lat/long coordinates
def haversine_distance(lat1, lon1, lat2, lon2):
    r = 6371
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
    res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return np.round(res, 2)

## Collect Users

In [3]:
def getSantaColomaTweets(start_date, end_date):
    df = pd.DataFrame()

    # Search Tweets with the various buildings and regions near Santa Coloma - Part 1
    c = twint.Config()
    c.Search = '\"Santa Coloma de Gramenet\" OR \"Santa Coloma\" OR \"Rambla San Sebastian\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 2
    c = twint.Config()
    c.Search = '\"Parque Fluvial del Besos\" OR \"Parc Fluvial del Besos\" OR \"Fluvial del Besos\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 3
    c = twint.Config()
    c.Search = '\"Parque Molinet\" OR \"Parc Molinet\" OR \"Molinet\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 4
    c = twint.Config()
    c.Search = '\"Plaza del Rellotge\" OR \"Plaça del Rellotge\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 5
    c = twint.Config()
    c.Search = '\"Parque Can Zam\" OR \"Parc Can Zam\" OR \"Can Zam\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 6
    c = twint.Config()
    c.Search = '\"Instituto Can Peixauet\" OR \"Institut Can Peixauet\" OR \"Can Peixauet\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 7
    c = twint.Config()
    c.Search = '\"Parque Gran Sol\" OR \"Parc Gran Sol\" OR \"Gran Sol\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 8
    c = twint.Config()
    c.Search = '\"Escuela Tanit\" OR \"Escola Tanit\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 9
    c = twint.Config()
    c.Search = '\"Instituto Terra Roja\" OR \"Institut Terra Roja\" OR \"Terra Roja\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 10
    c = twint.Config()
    c.Search = '\"Instituto Gassol\" OR \"Institut Gassol\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 11
    c = twint.Config()
    c.Search = '\"CAP Santa Rosa\" OR \"Cinto Verdaguer\" OR \"Nus de la Trinitat\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with the various buildings and regions near Santa Coloma - Part 12
    c = twint.Config()
    c.Search = '\"Mercado del Fondo\" OR \"Mercat del Fondo\" OR \"del Fondo\"'
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with location near 'Santa Coloma de Gramenet': North Side
    c = twint.Config()
    c.Geo = "41.46287400801948, 2.2028934732857177, 1km"
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    # Search Tweets with location near 'Santa Coloma de Gramenet': South Side
    c = twint.Config()
    c.Geo = "41.45039468429977, 2.212764002746006, 0.75km"
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    df = df.append(twint.storage.panda.Tweets_df)
    df = df.drop_duplicates(subset=['id'], ignore_index=True)
    
    return df

In [4]:
data = getSantaColomaTweets("2019-07-01", "2019-08-02") # effectively searches for dates 07/01 - 07/31

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping w

In [5]:
# collect users posting in native language of area
native_users = set(data.loc[data['language'].isin(['es', 'ca'])]['username'])
pickle.dump(native_users, open( "native_users.p", "wb" ) )
print("Num of native language users:", len(native_users))

# eliminate users posting in native language of area
users_with_nonnative_lang_tweets = set(data.loc[~data['language'].isin(['es', 'ca', 'und'])]['username'])
foreign_users = users_with_nonnative_lang_tweets - native_users
pickle.dump(foreign_users, open( "foreign_users.p", "wb" ) )
print("Num of nonnative language users:", len(foreign_users))

Num of native language users: 14570
Num of nonnative language users: 808


In [4]:
# read in pickle file
native_users = pickle.load( open( "native_users.p", "rb" ) )
foreign_users = pickle.load( open( "foreign_users.p", "rb") )

print("Num of native language users:", len(native_users))
print("Num of nonnative language users:", len(foreign_users))

Num of native language users: 14570
Num of nonnative language users: 808


# User Analysis

In [13]:
patterns = {
    "Santa Coloma de Gramanet" : [x.lower() for x in ["Santa Coloma de Gramenet", "Santa Coloma"]],
    "Fluvial del Besos" : [x.lower() for x in ["Parque Fluvial del Besos", "Parc Fluvial del Besos", "Fluvial del Besos", "rio.*besos|besos.*rio"]],
    "Parque Molinet" : [x.lower() for x in ["Parque Molinet", "Parc Molinet", "Molinet"]],
    "Plaza del Rellotge" : [x.lower() for x in ["Plaza del Rellotge", "Plaça del Rellotge"]],
    "Rambla San Sebastian" : [x.lower() for x in ["Rambla San Sebastian"]],
    "Parque Can Zam" : [x.lower() for x in ["Parque Can Zam", "Parc Can Zam", "Can Zam"]],
    "Instituto Can Peixauet" : [x.lower() for x in ["Instituto Can Peixauet", "Institut Can Peixauet", "Can Peixauet"]],
    "Parque Gran Sol" : [x.lower() for x in ["Parque Gran Sol", "Parc Gran Sol", "Gran Sol"]],
    "Escuela Tanit" : [x.lower() for x in ["Escuela Tanit", "Escola Tanit"]],
    "Instituto Terra Roja" : [x.lower() for x in ["Instituto Terra Roja", "Institut Terra Roja", "Terra Roja"]],
    "Instituto Gassol" : [x.lower() for x in ["Instituto Gassol", "Institut Gassol"]],
    "CAP Santa Rosa" : [x.lower() for x in ["CAP Santa Rosa"]],
    "Cinto Verdaguer" : [x.lower() for x in ["Cinto Verdaguer"]],
    "Mercado del Fondo" : [x.lower() for x in ["Mercado del Fondo", "Mercat del Fondo", "del Fondo"]],
    "Nus de la Trinitat" : [x.lower() for x in ["Nus de la Trinitat"]],
    "Macanet str" : [x.lower() for x in ["Maçanet str"]],
    "Iglesia Evangelica" : [x.lower() for x in ["Iglesia Evangelica", "Iglesia Esglesia"]]
}

## Foreign users

In [6]:
foreign_tweets_by_location = {
    "Santa Coloma de Gramanet" : pd.DataFrame(),
    "Fluvial del Besos" : pd.DataFrame(),
    "Parque Molinet" : pd.DataFrame(),
    "Plaza del Rellotge" : pd.DataFrame(),
    "Rambla San Sebastian" : pd.DataFrame(),
    "Parque Can Zam" : pd.DataFrame(),
    "Instituto Can Peixauet" : pd.DataFrame(),
    "Parque Gran Sol" : pd.DataFrame(),
    "Escuela Tanit" : pd.DataFrame(),
    "Instituto Terra Roja" : pd.DataFrame(),
    "Instituto Gassol" : pd.DataFrame(),
    "CAP Santa Rosa" : pd.DataFrame(),
    "Cinto Verdaguer" : pd.DataFrame(),
    "Mercado del Fondo" : pd.DataFrame(),
    "Nus de la Trinitat" : pd.DataFrame(),
    "Macanet str" : pd.DataFrame(),
    "Iglesia Evangelica" : pd.DataFrame()
}

In [7]:
i = 1
start = time.time()

# iterates through yearlong Tweets for the above selected users and filters users who have been associated with 
# Santa Coloma within 2 weeks 
for user in foreign_users:
    c = twint.Config()
    c.Username = user
    c.Since = "2019-01-01"
    c.Until = "2019-12-31"
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)

    df2 = twint.storage.panda.Tweets_df
    if (len(df2) > 0): 
        df2['tweet'] = df2['tweet'].str.lower()

        for k, location_df in foreign_tweets_by_location.items():
            pattern = '|'.join(patterns[k])

            tweets = df2[df2['tweet'].str.contains(pattern)]
            
            location_df = location_df.append(tweets)
            location_df = location_df.drop_duplicates(subset=['id'], ignore_index=True)
            foreign_tweets_by_location[k] = location_df
            
#             df2_tweets = df2_tweets.append(tweets)
#             df2_tweets = df2_tweets.drop_duplicates(subset=['id'], ignore_index=True)


#         if len(df2_tweets) > 0:
#             max_date = datetime.strptime(max(df2_tweets['date']), "%Y-%m-%d %H:%M:%S")
#             min_date = datetime.strptime(min(df2_tweets['date']), "%Y-%m-%d %H:%M:%S")

#             if max_date - min_date < timedelta(days=14):
#                 foreign_less_than_two_weeks.append(user)


#         df2_places = df2.loc[(df2['place'] != ''), ['username','place','date']]
#         if len(df2_places) > 0:
#             df2_places['coordinates'] = [x['coordinates'] for x in df2_places['place']]
#             # get distances to the two central points of Santa Coloma
#             df2_places['dist1'] = [haversine_distance(*x, 41.45039468429977, 2.212764002746006) for x in df2_places['coordinates']]
#             df2_places['dist2'] = [haversine_distance(*x, 41.46287400801948, 2.2028934732857177) for x in df2_places['coordinates']]
#             df2_places = df2_places[(df2_places['dist1'] < 0.75) | (df2_places['dist2'] < 1.0)]

#             if len(df2_places) > 0:
#                 max_date = datetime.strptime(max(df2_places['date']), "%Y-%m-%d %H:%M:%S")
#                 min_date = datetime.strptime(min(df2_places['date']), "%Y-%m-%d %H:%M:%S")

#                 if max_date - min_date < timedelta(days=14):
#                     foreign_places.append(user)

    print(str(math.floor(i * 100 / len(foreign_users))) + "% done. Time to completion: "
          + str(round(((time.time() - start) / i) * (len(foreign_users) - i), 0)) + " seconds.")
    i += 1
print("Total time of completion: " + str(time.time() - start) + " seconds.")

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 14404.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 13698.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 21091.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 51745.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 41616.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 37106.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 33069.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 3

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
8% done. Time to completion: 70876.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
8% done. Time to completion: 69754.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
8% done. Time to completion: 68706.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
8% done. Time to completion: 68265.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
8% done. Time to completion: 67463.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
8% done. Time to completion: 69976.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
8% done. Time to completion: 69013.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
9% done. Time to completion: 6

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
16% done. Time to completion: 54571.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
16% done. Time to completion: 54084.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
16% done. Time to completion: 53698.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
16% done. Time to completion: 53456.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
16% done. Time to completion: 53014.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
16% done. Time to completion: 52601.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
16% done. Time to completion: 52177.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
17% done. Time to compl

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
24% done. Time to completion: 42587.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
24% done. Time to completion: 43370.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
24% done. Time to completion: 43131.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
24% done. Time to completion: 42873.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
24% done. Time to completion: 42648.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
24% done. Time to completion: 42486.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
25% done. Time to completion: 42694.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
25% done. Time to compl

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
32% done. Time to completion: 42426.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
32% done. Time to completion: 42206.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
32% done. Time to completion: 41972.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
32% done. Time to completion: 41757.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
32% done. Time to completion: 41628.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
32% done. Time to completion: 41438.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
33% done. Time to completion: 42777.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
33% done. Time to compl

...

64% done. Time to completion: 23183.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
64% done. Time to completion: 23060.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
64% done. Time to completion: 22964.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
64% done. Time to completion: 22874.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
64% done. Time to completion: 22762.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
64% done. Time to completion: 22644.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
64% done. Time to completion: 22751.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
65% done. Time to completion: 22630.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
72% done. Time to completion: 17208.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
72% done. Time to completion: 17107.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
72% done. Time to completion: 17951.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
72% done. Time to completion: 17848.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
72% done. Time to completion: 17832.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
72% done. Time to completion: 17738.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
73% done. Time to completion: 17642.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
73% done. Time to compl

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
80% done. Time to completion: 12206.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
80% done. Time to completion: 12112.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
80% done. Time to completion: 12025.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
80% done. Time to completion: 11952.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
80% done. Time to completion: 11868.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
80% done. Time to completion: 11777.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
81% done. Time to completion: 11689.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
81% done. Time to compl

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
88% done. Time to completion: 7048.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
88% done. Time to completion: 6964.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
88% done. Time to completion: 6880.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
88% done. Time to completion: 6800.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
88% done. Time to completion: 6716.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
88% done. Time to completion: 6633.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
89% done. Time to completion: 6553.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
89% done. Time to completion: 

96% done. Time to completion: 2359.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
96% done. Time to completion: 2278.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
96% done. Time to completion: 2208.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
96% done. Time to completion: 2126.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
96% done. Time to completion: 2048.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
96% done. Time to completion: 1967.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
97% done. Time to completion: 1886.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
97% done. Time to completion: 1808.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets 

In [18]:
#original freq data
freq

{'Santa Coloma de Gramanet': 991,
 'Fluvial del Besos': 0,
 'Parque Molinet': 118,
 'Plaza del Rellotge': 5,
 'Rambla San Sebastian': 0,
 'Parque Can Zam': 76,
 'Instituto Can Peixauet': 7,
 'Parque Gran Sol': 329,
 'Escuela Tanit': 0,
 'Instituto Terra Roja': 2,
 'Instituto Gassol': 0,
 'CAP Santa Rosa': 0,
 'Cinto Verdaguer': 1,
 'Mercado del Fondo': 2061,
 'Nus de la Trinitat': 1}

In [25]:
foreign_freq

{'Santa Coloma de Gramanet': 597,
 'Fluvial del Besos': 16,
 'Parque Molinet': 107,
 'Plaza del Rellotge': 2,
 'Rambla San Sebastian': 0,
 'Parque Can Zam': 32,
 'Instituto Can Peixauet': 3,
 'Parque Gran Sol': 44,
 'Escuela Tanit': 0,
 'Instituto Terra Roja': 1,
 'Instituto Gassol': 0,
 'CAP Santa Rosa': 0,
 'Cinto Verdaguer': 1,
 'Mercado del Fondo': 1777,
 'Nus de la Trinitat': 0,
 'Macanet str': 0,
 'Iglesia Evangelica': 0}

In [28]:
foreign_user_count

{'Santa Coloma de Gramanet': 135,
 'Fluvial del Besos': 6,
 'Parque Molinet': 41,
 'Plaza del Rellotge': 1,
 'Rambla San Sebastian': 0,
 'Parque Can Zam': 7,
 'Instituto Can Peixauet': 2,
 'Parque Gran Sol': 37,
 'Escuela Tanit': 0,
 'Instituto Terra Roja': 1,
 'Instituto Gassol': 0,
 'CAP Santa Rosa': 0,
 'Cinto Verdaguer': 1,
 'Mercado del Fondo': 593,
 'Nus de la Trinitat': 0,
 'Macanet str': 0,
 'Iglesia Evangelica': 0}

In [6]:
for k, v in foreign_tweets_by_location.items():
    print(k + " - frequency:", len(v), "/ user count:", len(set(v["username"])))

Santa Coloma de Gramanet - frequency: 599 / user count: 137
Fluvial del Besos - frequency: 16 / user count: 6
Parque Molinet - frequency: 113 / user count: 42
Plaza del Rellotge - frequency: 2 / user count: 1
Rambla San Sebastian - frequency: 0 / user count: 0
Parque Can Zam - frequency: 32 / user count: 7
Instituto Can Peixauet - frequency: 3 / user count: 2
Parque Gran Sol - frequency: 44 / user count: 37
Escuela Tanit - frequency: 0 / user count: 0
Instituto Terra Roja - frequency: 1 / user count: 1
Instituto Gassol - frequency: 0 / user count: 0
CAP Santa Rosa - frequency: 0 / user count: 0
Cinto Verdaguer - frequency: 1 / user count: 1
Mercado del Fondo - frequency: 1798 / user count: 592
Nus de la Trinitat - frequency: 0 / user count: 0
Macanet str - frequency: 0 / user count: 0
Iglesia Evangelica - frequency: 0 / user count: 0


In [9]:
pickle.dump(foreign_tweets_by_location, open( "foreign_freq_tweets.p", "wb" ) )

In [5]:
foreign_tweets_by_location = pickle.load( open( "foreign_freq_tweets.p", "rb" ) )

## Native Users

In [7]:
native_tweets_by_location = {
    "Santa Coloma de Gramanet" : pd.DataFrame(),
    "Fluvial del Besos" : pd.DataFrame(),
    "Parque Molinet" : pd.DataFrame(),
    "Plaza del Rellotge" : pd.DataFrame(),
    "Rambla San Sebastian" : pd.DataFrame(),
    "Parque Can Zam" : pd.DataFrame(),
    "Instituto Can Peixauet" : pd.DataFrame(),
    "Parque Gran Sol" : pd.DataFrame(),
    "Escuela Tanit" : pd.DataFrame(),
    "Instituto Terra Roja" : pd.DataFrame(),
    "Instituto Gassol" : pd.DataFrame(),
    "CAP Santa Rosa" : pd.DataFrame(),
    "Cinto Verdaguer" : pd.DataFrame(),
    "Mercado del Fondo" : pd.DataFrame(),
    "Nus de la Trinitat" : pd.DataFrame(),
    "Macanet str" : pd.DataFrame(),
    "Iglesia Evangelica" : pd.DataFrame()
}
native_users_list = list(native_users)

In [23]:
print(native_users_list.index("BeaDeAlighieri"))
(native_users_list[:311] + native_users_list[312:800]).index("BeaDeAlighieri")

311


ValueError: 'BeaDeAlighieri' is not in list

In [11]:
def getNativeUserFrequencies(user_list):
    i = 1
    start = time.time()
    
    tweets_by_location = {
        "Santa Coloma de Gramanet" : pd.DataFrame(),
        "Fluvial del Besos" : pd.DataFrame(),
        "Parque Molinet" : pd.DataFrame(),
        "Plaza del Rellotge" : pd.DataFrame(),
        "Rambla San Sebastian" : pd.DataFrame(),
        "Parque Can Zam" : pd.DataFrame(),
        "Instituto Can Peixauet" : pd.DataFrame(),
        "Parque Gran Sol" : pd.DataFrame(),
        "Escuela Tanit" : pd.DataFrame(),
        "Instituto Terra Roja" : pd.DataFrame(),
        "Instituto Gassol" : pd.DataFrame(),
        "CAP Santa Rosa" : pd.DataFrame(),
        "Cinto Verdaguer" : pd.DataFrame(),
        "Mercado del Fondo" : pd.DataFrame(),
        "Nus de la Trinitat" : pd.DataFrame(),
        "Macanet str" : pd.DataFrame(),
        "Iglesia Evangelica" : pd.DataFrame()
    }
    # iterates through yearlong Tweets for the above selected users and filters users who have been associated with 
    # Santa Coloma within 2 weeks 
    for user in user_list:
        c = twint.Config()
        c.Username = user
        c.Since = "2019-01-01"
        c.Until = "2019-12-31"
        c.Pandas = True
        c.Hide_output = True
        twint.run.Search(c)

        df2 = twint.storage.panda.Tweets_df
        if (len(df2) > 0): 
            df2['tweet'] = df2['tweet'].str.lower()

            for k, location_df in tweets_by_location.items():
                pattern = '|'.join(patterns[k])

                tweets = df2[df2['tweet'].str.contains(pattern)]

#                 df2_tweets = df2_tweets.append(tweets)
#                 df2_tweets = df2_tweets.drop_duplicates(subset=['id'], ignore_index=True)
                location_df = location_df.append(tweets)
                location_df = location_df.drop_duplicates(subset=['id'], ignore_index=True)
                tweets_by_location[k] = location_df


#             if len(df2_tweets) > 0:
#                 max_date = datetime.strptime(max(df2_tweets['date']), "%Y-%m-%d %H:%M:%S")
#                 min_date = datetime.strptime(min(df2_tweets['date']), "%Y-%m-%d %H:%M:%S")

#                 if max_date - min_date < timedelta(days=14):
#                     less_than_two_weeks.append(user)


#             df2_places = df2.loc[(df2['place'] != ''), ['username','place','date']]
#             if len(df2_places) > 0:
#                 df2_places['coordinates'] = [x['coordinates'] for x in df2_places['place']]
#                 # get distances to the two central points of Santa Coloma
#                 df2_places['dist1'] = [haversine_distance(*x, 41.45039468429977, 2.212764002746006) for x in df2_places['coordinates']]
#                 df2_places['dist2'] = [haversine_distance(*x, 41.46287400801948, 2.2028934732857177) for x in df2_places['coordinates']]
#                 df2_places = df2_places[(df2_places['dist1'] < 0.75) | (df2_places['dist2'] < 1.0)]

#                 if len(df2_places) > 0:
#                     max_date = datetime.strptime(max(df2_places['date']), "%Y-%m-%d %H:%M:%S")
#                     min_date = datetime.strptime(min(df2_places['date']), "%Y-%m-%d %H:%M:%S")

#                     if max_date - min_date < timedelta(days=14):
#                         places.append(user)

        print(str(math.floor(i * 100 / len(user_list))) + "% done. Time to completion: "
              + str(round(((time.time() - start) / i) * (len(user_list) - i), 0)) + " seconds.")
        i += 1
    print("Total time of completion: " + str(time.time() - start) + " seconds.")
    
    return tweets_by_location

In [ ]:
modified_list_1 = native_users_list[:311] + native_users_list[312:800]
native_tweets_by_location1 = getNativeUserFrequencies(modified_list_1)
print(len(native_tweets_by_location1))
pickle.dump(native_tweets_by_location1, open( "native_freq_tweets1.p", "wb" ) )

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 2817.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 30446.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 23212.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 22863.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 21142.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 20927.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 18128.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
1% done. Time to completion: 19

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
8% done. Time to completion: 24384.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
8% done. Time to completion: 24036.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
8% done. Time to completion: 24000.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
8% done. Time to completion: 23675.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
8% done. Time to completion: 23413.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
8% done. Time to completion: 23361.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
9% done. Time to completion: 23219.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
9% done. Time to completion: 2

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
16% done. Time to completion: 19499.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
16% done. Time to completion: 19895.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
16% done. Time to completion: 19727.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
16% done. Time to completion: 19560.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
16% done. Time to completion: 19432.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
17% done. Time to completion: 19330.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
17% done. Time to completion: 19281.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
17% done. Time to compl

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
24% done. Time to completion: 18048.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
24% done. Time to completion: 18002.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
24% done. Time to completion: 17929.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
24% done. Time to completion: 17983.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
25% done. Time to completion: 17938.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
25% done. Time to completion: 17827.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
25% done. Time to completion: 17913.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
25% done. Time to compl

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
32% done. Time to completion: 16681.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
32% done. Time to completion: 16612.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
32% done. Time to completion: 16582.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
33% done. Time to completion: 16587.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
33% done. Time to completion: 16498.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
33% done. Time to completion: 16407.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
33% done. Time to completion: 16319.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
33% done. Time to compl

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
40% done. Time to completion: 17258.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
40% done. Time to completion: 17182.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
41% done. Time to completion: 17120.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
41% done. Time to completion: 17087.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
41% done. Time to completion: 17099.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
41% done. Time to completion: 17046.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
41% done. Time to completion: 16963.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
41% done. Time to compl

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
48% done. Time to completion: 15087.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
49% done. Time to completion: 15046.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
49% done. Time to completion: 15033.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
49% done. Time to completion: 14963.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
49% done. Time to completion: 14891.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
49% done. Time to completion: 14829.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
49% done. Time to completion: 14761.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
49% done. Time to compl

In [ ]:
for k, v in native_tweets_by_location1.items():
    print(k + " - frequency:", len(v), "/ user count:", len(set(v["username"])))

In [20]:
native_tweets_by_location2 = getNativeUserFrequencies(native_users_list[800:1600])
pickle.dump(native_tweets_by_location2, open( "native_freq_tweets2.p", "wb" ) )
for k, v in native_tweets_by_location2.items():
    print(k + " - frequency:", len(v), "/ user count:", len(set(v["username"])))

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 3211.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 2213.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 2125.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 6013.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 5237.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 9782.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 8570.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
1% done. Time to completion: 7766.0 s

KeyboardInterrupt: 

In [18]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[1600:2400], native_tweets_by_location)
time.sleep(1800)

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 1251.0 seconds.


KeyboardInterrupt: 

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[2400:3200], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[3200:4000], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[4000:4800], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[4800:5600], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[5600:6400], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[6400:7200], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[7200:8000], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[8000:8800], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[8800:9600], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[9600:10400], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[11200:12000], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[12000:12800], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[12800:13600], native_tweets_by_location)
time.sleep(1800)

In [ ]:
native_tweets_by_location = getNativeUserFrequencies(native_users_list[13600:14400], native_tweets_by_location)
time.sleep(1800)

In [16]:
native_tweets_by_location18 = getNativeUserFrequencies(native_users_list[14400:])
print(len(native_tweets_by_location18))
pickle.dump(native_tweets_by_location18, open( "native_freq_tweets18.p", "wb" ) )

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
0% done. Time to completion: 1541.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
1% done. Time to completion: 9841.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
1% done. Time to completion: 8731.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
2% done. Time to completion: 6806.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
2% done. Time to completion: 12360.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
3% done. Time to completion: 11118.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
4% done. Time to completion: 9916.0 seconds.
[!] No more data! Scraping will sto

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
38% done. Time to completion: 4054.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
38% done. Time to completion: 4008.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
39% done. Time to completion: 3921.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
40% done. Time to completion: 3967.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
40% done. Time to completion: 3894.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
41% done. Time to completion: 3805.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
41% done. Time to completion: 3721.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
42% done. Time to completion: 

75% done. Time to completion: 1873.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
76% done. Time to completion: 1829.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
77% done. Time to completion: 1779.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
77% done. Time to completion: 1726.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
78% done. Time to completion: 1670.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
78% done. Time to completion: 1642.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
79% done. Time to completion: 1608.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
80% done. Time to completion: 1552.0 seconds.
[!] No more data! Scraping will stop now.
found 0 deleted tweets 

In [19]:
print(len(native_tweets_by_location))

17


# Location Based

In [ ]:
c = twint.Config()
c.Search = '\"Santa Coloma de Gramenet\" OR \"Santa Coloma\" OR \"Rambla San Sebastian\"'
c.Since = "2019-01-01"
c.Until = "2019-01-03"
c.Pandas = True
c.Hide_output = True
twint.run.Search(c)

df = twint.storage.panda.Tweets_df